In [1]:
# Library imports
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Prunning Youtube Network for Analysis

In [3]:
youtubeG = nx.read_edgelist("data/com-youtube.ungraph.txt", create_using = nx.Graph, nodetype = int)

The next cell will make the youtube network much smaller so we can 

In [12]:

# Obtains list of degrees that have degree less than 3
degree_Thres = 6
remove = [node for node,degree in dict(youtubeG.degree()).items() if  degree < degree_Thres]

# Removes nodes with small degree and creates sparse matrix
youtubeG.remove_nodes_from(remove)
nx.write_edgelist(youtubeG,'youtube_sparse.txt', data=False)


# Cluster Analysis

In [3]:
wikiG = nx.read_edgelist("input/wiki-Vote.txt", create_using = nx.DiGraph, nodetype = int)
collabG = nx.read_edgelist("input/ca-GrQc.txt", create_using = nx.Graph, nodetype = int)
p2pG = nx.read_edgelist("input/p2p-Gnutella08.txt", create_using = nx.Graph, nodetype = int)
fbG = nx.read_edgelist("input/facebook_combined.txt", create_using = nx.Graph, nodetype = int)

mean_nodes = (wikiG.number_of_nodes() + collabG.number_of_nodes() + p2pG.number_of_nodes() + fbG.number_of_nodes()) / 4
mean_edges = (wikiG.number_of_edges() + collabG.number_of_edges() + p2pG.number_of_edges() + fbG.number_of_edges()) / 4
pd.DataFrame({"Graph": ["Wikipedia", "Collaborations", "Gnutella", "Facebook", "Mean(G)"],
    "Node Count": [wikiG.number_of_nodes(), collabG.number_of_nodes(), p2pG.number_of_nodes(), fbG.number_of_nodes(), mean_nodes],
    "Edge Count": [wikiG.number_of_edges(), collabG.number_of_edges(), p2pG.number_of_edges(), fbG.number_of_edges(), mean_edges]})

,Graph,Node Count,Edge Count
0,Wikipedia,7115.00,103689.0
1,Collaborations,5242.00,14496.0
2,Gnutella,6301.00,20777.0
3,Facebook,4039.00,88234.0
4,Mean(G),5674.25,56799.0


In [17]:
wikiCluster = np.loadtxt("output/wiki-Vote.metis.part.100")
len(wikiCluster)

7115